In [1]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd
from boto import get_square_secret
import uuid

# import os
import datetime as dt
import pickle

In [2]:
df=pd.read_csv('invoice_csv.csv')
df

,Players,amount,email
0,Afnan Khan,5.0,afnanblue9@outlook.com
1,Aidan Byrne,15.0,byrneyboii2@gmail.com
2,Alex Dockerill,5.0,dockerillalex@gmail.com
3,Babajide Onibada,10.0,jideonibada@hotmail.com
4,Dix Leong,10.0,dixleong@gmail.com
5,Dom Robbins,5.0,dominicrobbins2001@gmail.com
6,Ellis Forrester,5.0,ellisforrester1@gmail.com
7,Emmanuel oduro,10.0,Emmanueloduro123@outlook.com
8,Harold Kiyingi,15.0,hkiyingi99@gmail.com
9,Ismail Wall,10.0,ismailwall753@gmail.com


In [3]:
df['email']=df['email'].apply(lambda x: x.strip())

In [4]:
get_square_secret()

'EAAAFO_dqOOnvRZCLDvOtKfBGznlxPy5zb4anviDPIJBjXjXGkmo5d6w1jxoxtCn'

In [5]:
client = Client(
    access_token=get_square_secret(),
    environment='production')

In [6]:
customers_api = client.customers

In [7]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8
9
10


In [8]:
customer_df=pd.DataFrame(data=[{'email':cust, 'id':customers[cust]} for cust in customers])
customer_df

,email,id
0,None,7QMAV9NTQRNDMVW6V81PKKTKZ4
1,aaronrose1981@gmail.com,TJ6J476K0X5HFCDCXEA8QEN84M
2,aaron.s.c.s@hotmail.co.uk,0VBREWQGYS5G3D5764XW4TZH98
3,83251863.mail@waitrose.co.uk,MPWDP86M290BZ5VWVGK51JMJD8
4,absstacks98@gmail.com,CDWK490YN7HBA3Z3CRRTS843EW
...,...,...
664,oaoviosu@gmail.com,KRVP85KPCH7Y77HPR47JN3WFPC
665,zac.rabanalhunt@gmail.com,224DCGABNQZBAP69A4JM3VBSQ8
666,zachary_mark_williams@yahoo.co.uk,TF1ZAX80D09ST3J7PK4PGY0RN8
667,zakriyagul23@gmail.com,V295YNGJSD4235PXWGMEVKK088


In [9]:
mg = pd.merge(left =df, right = customer_df, how='left', on=['email'])

In [10]:
not_created = mg[mg['id'].isnull()].copy()
not_created

,Players,amount,email,id
19,Oluwatobi Ajetunmobi,5.0,oluwatobiajetunmobi@yahoo.com,NaN


In [11]:
for i, row in not_created.iterrows():
    print(row)

    first_name=row['Players'].split(' ')[0]
    surname=row['Players'].split(' ')[0]

    email=row['email']

    body = {
    'given_name': first_name,
    'family_name': surname,
    'email_address':email.strip(),
    
    }

    result = customers_api.create_customer(body)
    print(result)

Players             Oluwatobi Ajetunmobi
amount                               5.0
email      oluwatobiajetunmobi@yahoo.com
id                                   NaN
Name: 19, dtype: object
<ApiResponse {"customer": {"id": "8DDMF6VXNV5J9WT2R9SBFF0MKG","created_at": "2024-09-24T09:38:53.075Z","updated_at": "2024-09-24T09:38:53Z","given_name": "Oluwatobi","family_name": "Oluwatobi","email_address": "oluwatobiajetunmobi@yahoo.com","preferences": {"email_unsubscribed": false},"creation_source": "THIRD_PARTY","version": 0}}
>


In [12]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8
9
10


In [13]:
customer_df=pd.DataFrame(data=[{'email':cust, 'id':customers[cust]} for cust in customers])
customer_df

,email,id
0,None,7QMAV9NTQRNDMVW6V81PKKTKZ4
1,aaronrose1981@gmail.com,TJ6J476K0X5HFCDCXEA8QEN84M
2,aaron.s.c.s@hotmail.co.uk,0VBREWQGYS5G3D5764XW4TZH98
3,83251863.mail@waitrose.co.uk,MPWDP86M290BZ5VWVGK51JMJD8
4,absstacks98@gmail.com,CDWK490YN7HBA3Z3CRRTS843EW
...,...,...
665,oaoviosu@gmail.com,KRVP85KPCH7Y77HPR47JN3WFPC
666,zac.rabanalhunt@gmail.com,224DCGABNQZBAP69A4JM3VBSQ8
667,zachary_mark_williams@yahoo.co.uk,TF1ZAX80D09ST3J7PK4PGY0RN8
668,zakriyagul23@gmail.com,V295YNGJSD4235PXWGMEVKK088


In [14]:
mg = pd.merge(left =df, right = customer_df, how='left', on=['email'])
mg

,Players,amount,email,id
0,Afnan Khan,5.0,afnanblue9@outlook.com,N76SMZPS5JB3RH94JBD59P3WXM
1,Aidan Byrne,15.0,byrneyboii2@gmail.com,ZSBNEMP88D61Z4V4FEM51RHD0W
2,Alex Dockerill,5.0,dockerillalex@gmail.com,D94PE6KJF55XQAC3KMJ0GMBGNR
3,Babajide Onibada,10.0,jideonibada@hotmail.com,CZ3CB8ATNH58BAN5HKEG3W0ESW
4,Dix Leong,10.0,dixleong@gmail.com,016HP9A0XX5SZE6CQ0W6KMXXKW
5,Dom Robbins,5.0,dominicrobbins2001@gmail.com,HEM047TWM96F5KV5B1J7BGBGTM
6,Ellis Forrester,5.0,ellisforrester1@gmail.com,RQ06G4N4SDZSKB4AX0ZP1RNSQC
7,Emmanuel oduro,10.0,Emmanueloduro123@outlook.com,KMTCYHZC712KF5QKQJYG5R1KPW
8,Harold Kiyingi,15.0,hkiyingi99@gmail.com,XVVDQVHP8TWJ9B88DFHZ6BZFFC
9,Ismail Wall,10.0,ismailwall753@gmail.com,2GK979CYQQ3GKNF7VN2GMR7KTG


In [15]:
orders_api = client.orders

new_orders = []

for i, row in mg.iterrows():
    print(i)
    amount = row['amount'] 
    body = {
        'order': {
            'location_id': 'SBBJSGR24YHYB',
            'reference_id': f'{uuid.uuid1()}',
            'line_items': [
                
                {
                    'quantity': '1',
                    'catalog_object_id': 'FLDAWYXJJW5L5NGKQFLVU3NC',
                    'base_price_money': {
                        'amount': amount*100,
                        'currency': 'GBP'
                    }
                }
            ]
        },
        'idempotency_key': f'{uuid.uuid1()}'
    }
    # print(body)
    
    result = orders_api.create_order(body)
    # print(result)
    obj_id=result.body['order']['id']

    new_orders.append({'email':row['email'], 'obj_id':obj_id})
    # # print(result)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [16]:
order_mg= pd.merge(left = mg, right=pd.DataFrame(new_orders), how= 'left', on = 'email')

order_mg

,Players,amount,email,id,obj_id
0,Afnan Khan,5.0,afnanblue9@outlook.com,N76SMZPS5JB3RH94JBD59P3WXM,f0a46N7GBFgqgQe2k7phZoA7kyXZY
1,Aidan Byrne,15.0,byrneyboii2@gmail.com,ZSBNEMP88D61Z4V4FEM51RHD0W,Bh25t660QDdhTf8pSPOXab6HLYGZY
2,Alex Dockerill,5.0,dockerillalex@gmail.com,D94PE6KJF55XQAC3KMJ0GMBGNR,bk9xI9SNp2b0hUTKMRgfip8OWnGZY
3,Babajide Onibada,10.0,jideonibada@hotmail.com,CZ3CB8ATNH58BAN5HKEG3W0ESW,DUxCh0AaharlK1cOIQ2dQITt9wLZY
4,Dix Leong,10.0,dixleong@gmail.com,016HP9A0XX5SZE6CQ0W6KMXXKW,53JWC29HHIKemCb16UuaAFvRoOOZY
5,Dom Robbins,5.0,dominicrobbins2001@gmail.com,HEM047TWM96F5KV5B1J7BGBGTM,tLchha0I9aoXVmYcGegw2TmNmQfZY
6,Ellis Forrester,5.0,ellisforrester1@gmail.com,RQ06G4N4SDZSKB4AX0ZP1RNSQC,hRW73lK4vee3NbNQPE54jMpy7cIZY
7,Emmanuel oduro,10.0,Emmanueloduro123@outlook.com,KMTCYHZC712KF5QKQJYG5R1KPW,pLhZZQUuYA47N5C7p4liiWkZNnOZY
8,Harold Kiyingi,15.0,hkiyingi99@gmail.com,XVVDQVHP8TWJ9B88DFHZ6BZFFC,n0WVY7u8xybEcyxzWfTh5pgd2PEZY
9,Ismail Wall,10.0,ismailwall753@gmail.com,2GK979CYQQ3GKNF7VN2GMR7KTG,9Ty4LudM4MPG8PpPxbZqY2LOUuQZY


In [17]:
invoices_api = client.invoices

In [18]:
due_at =  str(dt.datetime.now().date())
hour_due=dt.datetime.now().hour+1
due_at+f'T{hour_due}:00:00Z',

('2024-09-24T11:00:00Z',)

In [19]:
invoices = []

for i, row in order_mg[order_mg['id'].notnull()].iterrows():
    order_id = row['obj_id']
    cust_id = row['id']
    invoice_number = str(dt.datetime.now().date()) +f"_{row['obj_id']}_1"

    

    title = 'Total Outstanding Training Subs'

        
    body = {
    'invoice': {
        'location_id': 'SBBJSGR24YHYB',
        'order_id': f'{order_id}',
        'primary_recipient': {
            'customer_id': f'{cust_id}'
        },
        'payment_requests': [
            {
                'request_type': 'BALANCE',
                'due_date': due_at,
                'tipping_enabled': True,
                'automatic_payment_source': 'NONE',
                'reminders': [
                    {
                        'relative_scheduled_days': -1,
                        'message': 'Your Invoice is due'
                    }
                ]
            }
        ],
        'delivery_method': 'EMAIL',
        'invoice_number': invoice_number,
        'title': title,
        'description': title,
        'scheduled_at': due_at+f'T{hour_due}:00:00Z',
        'accepted_payment_methods': {
            'card': True,
            'square_gift_card': False,
            'bank_account': False,
            'buy_now_pay_later': False,
            'cash_app_pay': False
        },
        'sale_or_service_date': str(dt.datetime.now().date()),
        'store_payment_method_enabled': False
    },
    'idempotency_key': f'{uuid.uuid1()}'
    }

    result = invoices_api.create_invoice(body)
    print(row['email'])
    print(result)
    invidid=result.body['invoice']['id']
    invoices.append(invidid)

afnanblue9@outlook.com
<ApiResponse {"invoice":{"id":"inv:0-ChAxeaDeHTEJRp6bRvaU7Vp2EIMP","version":0,"location_id":"SBBJSGR24YHYB","order_id":"f0a46N7GBFgqgQe2k7phZoA7kyXZY","payment_requests":[{"uid":"0ebdad02-b956-4710-8d1a-21a2dac91dea","request_type":"BALANCE","due_date":"2024-09-24","tipping_enabled":true,"computed_amount_money":{"amount":500,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"0a524565-1c82-4261-93e5-911c5e721a5f","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"PENDING"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"N76SMZPS5JB3RH94JBD59P3WXM","given_name":"Afnan","family_name":"Khan","email_address":"afnanblue9@outlook.com","address":{"country":"GB"}},"invoice_number":"2024-09-24_f0a46N7GBFgqgQe2k7phZoA7kyXZY_1","title":"Total Outstanding Training Subs","description":"Total Outstanding Training Subs","scheduled_at":"2024-09-24T11:00:00Z","status":"DRAFT","timezone":

In [20]:
invoices

['inv:0-ChAxeaDeHTEJRp6bRvaU7Vp2EIMP',
 'inv:0-ChCB6HkCsEYsoU6XovGjcP4xEIMP',
 'inv:0-ChBQ9hZXDnysryo_pIUYp8_2EIMP',
 'inv:0-ChCsjwV1W6LCZWCuk1WiEb_0EIMP',
 'inv:0-ChDxlC7d-1CkecY7Qsg3BKDZEIMP',
 'inv:0-ChBAWncWRK1v2GsyHG2YVRpZEIMP',
 'inv:0-ChDpRat7H9Kl0CtXndIoHJbhEIMP',
 'inv:0-ChClvNZEmefaCoL-RVrND2eTEIMP',
 'inv:0-ChAyfw6Uo_YQzUgYTRCnvQxGEIMP',
 'inv:0-ChDtjnSBApMPDAsroxiiDWdbEIMP',
 'inv:0-ChD09q5hfBefFtqS96-wwBvLEIMP',
 'inv:0-ChDLK1avFPhi80--9rSehTdFEIMP',
 'inv:0-ChBStuNFt8rVAErfI6FzXxxsEIMP',
 'inv:0-ChCPybs8WNN8u0wPeursosorEIMP',
 'inv:0-ChDdfxWMG3DatIhxuGk3b4vREIMP',
 'inv:0-ChDKtbp4L2JeUMNGIYdGZxWCEIMP',
 'inv:0-ChA6U31AUOUSQV-IYt_aq-2EEIMP',
 'inv:0-ChDe2ZpXKVjxJUv5Tll1Rv4cEIMP',
 'inv:0-ChCQNn8rfL7tg4qyImaTxVyCEIMP',
 'inv:0-ChBAb1cjInRpQmfqD7RBa6X3EIMP',
 'inv:0-ChCtRUc0-xc86yi19WolernuEIMP',
 'inv:0-ChCVk93Fvq_YcmHx7bvfMDnKEIMP',
 'inv:0-ChDKIEWeQKeF4cucinsiDyC8EIMP']

In [21]:
for invidid in invoices:

    body = {
        'version': 0,
        'idempotency_key': f'{invidid}'
    }

    result = invoices_api.publish_invoice(
        invidid,
        body
    )
    print(result)

<ApiResponse {"invoice":{"id":"inv:0-ChAxeaDeHTEJRp6bRvaU7Vp2EIMP","version":1,"location_id":"SBBJSGR24YHYB","order_id":"f0a46N7GBFgqgQe2k7phZoA7kyXZY","payment_requests":[{"uid":"0ebdad02-b956-4710-8d1a-21a2dac91dea","request_type":"BALANCE","due_date":"2024-09-24","tipping_enabled":true,"computed_amount_money":{"amount":500,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"0a524565-1c82-4261-93e5-911c5e721a5f","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"NOT_APPLICABLE"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"N76SMZPS5JB3RH94JBD59P3WXM","given_name":"Afnan","family_name":"Khan","email_address":"afnanblue9@outlook.com","address":{"country":"GB"}},"invoice_number":"2024-09-24_f0a46N7GBFgqgQe2k7phZoA7kyXZY_1","title":"Total Outstanding Training Subs","description":"Total Outstanding Training Subs","scheduled_at":"2024-09-24T11:00:00Z","status":"SCHEDULED","timezone":"Europe/Lond